In [ ]:
!pip install transformers accelerate

In [ ]:
# Clone your GitHub repo into Kaggle
!git clone https://github.com/abhikad292/abhinav_sample.git

In [ ]:
%cd abhinav_sample/llm-prompt-comparison

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # suppress TensorFlow/XLA warnings
# Importing the transformers library classes
# AutoTokenizer -> used to convert human text into tokens (numbers) the model can understand
# AutoModelForCausalLM -> loads a pre-trained language model capable of generating text (Causal LM = causal language model)
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

# Step 1: Choose the model you want to use
# Here we are using "Gemma 7B IT", an open-source LLM from Google
model_name = "/kaggle/input/gemma/transformers/2b-it/3"

# Step 2: Load the tokenizer
# A tokenizer converts text (your input prompt) into numbers (tokens) that the model can understand
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Step 3: Load the model into memory
# AutoModelForCausalLM loads a causal language model (good for text generation tasks)
model = AutoModelForCausalLM.from_pretrained(model_name,device_map="auto", offload_folder="offload")

In [ ]:
# Step 4: Define query function (fixed temperature issue)
def query_model(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    # Define generation configuration
    gen_config = GenerationConfig(
        max_new_tokens=2048,
        temperature=0.7,  # adjust randomness
        do_sample=True     # required if using temperature
    )
    
    outputs = model.generate(**inputs, generation_config=gen_config)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()
# Step 5: Load universal prompt from GitHub
with open("prompts/universal_prompt.txt") as f:
    universal_prompt = f.read()

# Step 6: Add your test rules
rule1 = """Given the condition: If FICO > 750 and FICO <= 900 and NO_INQ >= 2 and NO_INQ <= 99 then DECISION_CD is 'Declined' and DECISION_DESC is 'INQUIRIES'.
Generate all combinations of min, mid, max for FICO and NO_INQ within these bounds.
Output in format: (FICO, NO_INQ, DECISION_CD, DECISION_DESC)"""

# Step 7: Run the model
output1 = query_model(rule1)
print("Input 1 Output:\n", output1)
with open("open_source_input1.json", "w") as f:
    f.write(output1)

rule2="""Given the condition: If Income > 50000 and Debt_To_Income < 0.4 then Loan_status is Approved and Comment is 'Low Risk'.
Generate all the possible combination where loan status is approved and comment is low risk.
Output in format: (Income, Debt_To_Income, Loan_status, comment)"""

# Step 7: Run the model
output2 = query_model(rule2)
print("Input 2 Output:\n", output2)
with open("open_source_input2.json", "w") as f:
    f.write(output2)